## Exploring the Stack Overflow 2023 Developer Survey

### Introduction
This notebook represents my attempt in exploring the Stack Overflow 2023 Developer Survey data using Python. The primary aim is to analyze factors that predict developers' total compensation and to uncover additional insights as the data reveals them. Initially, we begin with a focused question on compensation, but as the analysis progresses, other interesting questions emerge.

### Data Source
The data for this analysis was obtained from Stack Overflow's annual developer survey for 2023, which is publicly available on Kaggle at the following link: [Stack Overflow 2023 Developer Survey](https://www.kaggle.com/datasets/stackoverflow/stack-overflow-2023-developers-survey).



In [833]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Reset all pandas display options to their default values
pd.reset_option('all')

import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_dark" 


# Machine learning
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


C:\Users\DanCohen\AppData\Local\Temp\ipykernel_17820\3815561866.py:6: FutureWarning:

data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.

C:\Users\DanCohen\AppData\Local\Temp\ipykernel_17820\3815561866.py:6: FutureWarning:

use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.



### Exploring and Preparing the data

Starting off by loading the data and schema files and displaying our dataframe

In [834]:
df = pd.read_csv('survey_results_public.csv')
schema_df = pd.read_csv('survey_results_schema.csv')
df.head(3)

,ResponseId,Q120,MainBranch,Age,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,...,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,ProfessionalTech,Industry,SurveyLength,SurveyEase,ConvertedCompYearly
0,1,I agree,None of these,18-24 years old,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,I agree,I am a developer by profession,25-34 years old,"Employed, full-time",Remote,Hobby;Contribute to open-source projects;Boots...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;Friend or fam...,Formal documentation provided by the owner of ...,...,1-2 times a week,10+ times a week,Never,15-30 minutes a day,15-30 minutes a day,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",Appropriate in length,Easy,285000.0
2,3,I agree,I am a developer by profession,45-54 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby;Professional development or self-paced l...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;On the job tr...,Formal documentation provided by the owner of ...,...,6-10 times a week,6-10 times a week,3-5 times a week,30-60 minutes a day,30-60 minutes a day,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",Appropriate in length,Easy,250000.0


 I use a function to display data types, cardinality, and missing values for each column. This helps me better understading the structure of the dataset and also decide how to handle each of the columns. 

In [835]:
def column_info(df):
    """
    Generate a summary of column information for a DataFrame.

    This function collects the following information for each column:
    - Column name
    - Data type
    - Number of missing values
    - Percentage of missing values
    - Cardinality (number of unique values)

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: A DataFrame containing the column information with columns:
            - 'Column'
            - 'Dtype'
            - 'Missing Values'
            - 'missing_values_pct'
            - 'Cardinality'
    """
    info = []
    for column in df.columns:
        dtype = df[column].dtype
        missing_values = df[column].isnull().sum()
        missing_values_pct = missing_values / df.shape[0]
        cardinality = df[column].nunique()
        info.append((column, dtype, missing_values, missing_values_pct, cardinality))
    
    info_df = pd.DataFrame(info, columns=['Column', 'Dtype', 'Missing Values', 'missing_values_pct', 'Cardinality'])
    return info_df


info_df = column_info(df).sort_values(by='Cardinality',ascending=False)
info_df.head(50)

,Column,Dtype,Missing Values,missing_values_pct,Cardinality
0,ResponseId,int64,0,0.000000,89184
50,SOAI,object,41326,0.463379,43061
31,ToolsTechHaveWorkedWith,object,11300,0.126704,33133
21,LanguageHaveWorkedWith,object,2044,0.022919,32641
22,LanguageWantToWorkWith,object,8475,0.095028,29602
32,ToolsTechWantToWorkWith,object,20869,0.233999,27456
33,NEWCollabToolsHaveWorkedWith,object,3320,0.037226,21262
27,WebframeHaveWorkedWith,object,22246,0.249439,15144
28,WebframeWantToWorkWith,object,32443,0.363776,14620
34,NEWCollabToolsWantToWorkWith,object,12535,0.140552,13659


Using the previous output and schema file, we observed that most columns are of type 'object', which typically means they contain strings. Additionally, many columns exhibit very high cardinality. This is primarily because respondents were allowed to provide multiple answers, which are represented in the dataset as strings separated by semicolons (';').

For instance, there are 29,602 unique values for the LanguageWantToWorkWith column. This is because respondents could specify multiple languages. For example, one respondent might list C#; Java, another might list only Java, and a third might list only C#. Consequently, we have three unique values in the dataset.

In [836]:
def print_questions_text(df, qids):
    with pd.option_context('display.max_colwidth', None):
        for id in qids:
                print(df[df['qname'] == id]['question'])

        

print_questions_text(schema_df, ['SOAI'])

45    Artificial Intelligence (AI) tools have gained prominence recently across industries. The following question asks for your perspective on Stack Overflow using AI technology to improve the current experience, for example by using AI to suggest better titles for your question. For your response, consider these sorts of improvements to the experience with/on Stack Overflow and not the presence or absence of content generated by AI and posted on Stack Overflow. <br><br>What is your opinion on Stack Overflow using AI tools to improve the current experience? What could AI help with most to improve Stack Overflow?
Name: question, dtype: object


Here are two columns I want to remove immediately:

1. SOAI - Open responses often have high variability and cardinality, meaning there are many unique answers. This can make it difficult to handle.
2. ResponseId - This is just the rresponse ID and does not hold any meanigful information

In [837]:
# Drop unwanted features
excluded_features = ['ResponseId','SOAI']
df = df.drop(excluded_features,axis=1)
df.columns


Index(['Q120', 'MainBranch', 'Age', 'Employment', 'RemoteWork',
       'CodingActivities', 'EdLevel', 'LearnCode', 'LearnCodeOnline',
       'LearnCodeCoursesCert', 'YearsCode', 'YearsCodePro', 'DevType',
       'OrgSize', 'PurchaseInfluence', 'TechList', 'BuyNewTool', 'Country',
       'Currency', 'CompTotal', 'LanguageHaveWorkedWith',
       'LanguageWantToWorkWith', 'DatabaseHaveWorkedWith',
       'DatabaseWantToWorkWith', 'PlatformHaveWorkedWith',
       'PlatformWantToWorkWith', 'WebframeHaveWorkedWith',
       'WebframeWantToWorkWith', 'MiscTechHaveWorkedWith',
       'MiscTechWantToWorkWith', 'ToolsTechHaveWorkedWith',
       'ToolsTechWantToWorkWith', 'NEWCollabToolsHaveWorkedWith',
       'NEWCollabToolsWantToWorkWith', 'OpSysPersonal use',
       'OpSysProfessional use', 'OfficeStackAsyncHaveWorkedWith',
       'OfficeStackAsyncWantToWorkWith', 'OfficeStackSyncHaveWorkedWith',
       'OfficeStackSyncWantToWorkWith', 'AISearchHaveWorkedWith',
       'AISearchWantToWorkWith', 

Our target is 'CovertedCompYearly' which simply means Total Yearly Compensation. 
I drop the all rows with missing missing compensation values.

In [838]:
# Dropping na compensation rows
print(f'Original length: {len(df)}')
df = df.dropna(subset=['ConvertedCompYearly'])

# Printing new dataframe length
print(f'After drop length: {len(df)}')


Original length: 89184
After drop length: 48019


Filtering a DataFrame based on specific criteria, is essential for focusing on relevant data, ensuring consistency, and improving our model performance. It also allows us to work with a comparable subset of data, reducing noise and variability. In our case, I choose to focus on full time developers in the US.

In [839]:
# Filtering the data to only full-time developers in the US
df = df[
    (df['Currency'] == 'USD	United States dollar') &
    (df['Country'] == 'United States of America') &
    (df['MainBranch'] == 'I am a developer by profession') &
    (df['Employment'] == 'Employed, full-time')]
    

# Drop the filtered columns
df = df.drop(['Currency', 'Country', 'MainBranch', 'Employment'], axis=1)


Another thing I noticed is that the YearsCode and YearsCodePro columns are of type 'object' but contain numerical values. Since we will later normalize these features using the StandardScaler, I will convert them to numeric types.

In [840]:
# Convert to numeric types
df['YearsCode'] = pd.to_numeric(df['YearsCode'], errors='coerce')
df['YearsCodePro'] = pd.to_numeric(df['YearsCodePro'], errors='coerce')

Next, we'll remove outliers 

In [841]:
# Calculate the 1st and 99th percentiles for the 'CompTotal' column
lower_bound = df['CompTotal'].quantile(0.01)
upper_bound = df['CompTotal'].quantile(0.99)

# Filter the DataFrame to include only rows within these bounds
df = df[(df['ConvertedCompYearly'] >= lower_bound) & (df['CompTotal'] <= upper_bound)]

We now visualize the total compensation distribution

In [842]:
# Create a histogram with Plotly
fig = px.histogram(
    df,
    x='ConvertedCompYearly',
    title='Salary Distribution',
    labels={'ConvertedCompYearly': 'Salary'},
    nbins=20,  
    color_discrete_sequence=['skyblue'],
    width=1000
)

# Update the layout for better appearance
fig.update_layout(
    xaxis_title='Compensation',
    yaxis_title='Frequency',
    bargap=0.1,  # Gap between bars
    showlegend=False
)

fig.show()

That's nice. We see the much expected right-skew.
Alright, we are ready for questiosn number one, but important note before that:

<b> Note: Each question pertains to respondents who work as full-time developers in the US and participated in the Stack Overflow Survey. </b>

### 1. Which developer type has the highest and lowest compensation?



In [843]:
# Group by 'DevType' and calculate mean and count
dev_type_sal_df = df.groupby('DevType').agg(
    MeanComp=('ConvertedCompYearly', 'mean'),
    MedianComp=('ConvertedCompYearly', 'median'),
    Count=('DevType', 'size')
).reset_index()

dev_type_sal_df['MeanComp'] = dev_type_sal_df['MeanComp'].apply(lambda x: f'${x:,.1f}')
dev_type_sal_df['MedianComp'] = dev_type_sal_df['MedianComp'].apply(lambda x: f'${x:,.1f}')



# Drop rows where the count is lower than 50
dev_type_sal_df = dev_type_sal_df[dev_type_sal_df['Count'] >= 50]

# Display top ten
dev_type_sal_df.sort_values(by='MedianComp', ascending=False).head(10)


,DevType,MeanComp,MedianComp,Count
30,"Senior Executive (C-Suite, VP, etc.)","$238,210.4","$225,000.0",149
9,Developer Experience,"$219,935.7","$210,000.0",63
21,Engineering manager,"$203,569.7","$192,500.0",329
20,"Engineer, site reliability","$204,790.6","$190,000.0",81
2,Cloud infrastructure engineer,"$199,086.0","$182,500.0",192
24,Other (please specify):,"$187,562.2","$180,000.0",252
11,"Developer, back-end","$184,616.9","$165,000.0",1820
19,"Engineer, data","$171,181.3","$163,300.0",212
17,"Developer, mobile","$181,111.1","$162,000.0",249
27,Research & Development role,"$179,035.4","$161,500.0",146


In [844]:
dev_type_sal_df.sort_values(by='MedianComp', ascending=False).tail(10)

,DevType,MeanComp,MedianComp,Count
17,"Developer, mobile","$181,111.1","$162,000.0",249
27,Research & Development role,"$179,035.4","$161,500.0",146
16,"Developer, game or graphics","$188,018.7","$160,000.0",82
4,Data scientist or machine learning specialist,"$182,818.4","$157,500.0",182
7,DevOps specialist,"$173,011.7","$155,000.0",177
13,"Developer, embedded applications or devices","$159,232.4","$140,000.0",331
15,"Developer, full-stack","$149,324.2","$140,000.0",3774
14,"Developer, front-end","$151,651.8","$139,010.0",574
12,"Developer, desktop or enterprise applications","$144,407.8","$130,000.0",496
10,"Developer, QA or test","$141,072.2","$128,596.0",65


For most people in the tech industry, these results are not very surprising. However, I was surprised to see full-stack engineers ranking so low on the list in terms of compensation. One of the more unexpected findings is that developer experience engineers have a median compensation of $210,000! I guess the next step for me, is to figure out what is a developer experience engineer.


I want to understand what factors contribute to the variance in compensation within a specific developer type. I'll focus on full-stack engineers since they have the highest number of respondents in the survey.



### 2. What factors contribute to the variance in componsation for full-stack engineers?


For this analysis, I will use a simple correlation matrix and linear regression to gain insights into some of the key factors. This approach requires some additional data preparation before we can begin the actual analysis.

In [845]:
# Filtering by full-stack engineers
df = df[df['DevType'] == 'Developer, full-stack']
df.shape

(3774, 78)

I'm going to use a helper column here for compensation ranking(1-5)

In my experience ranking reduce complexity and helps focus on major patterns rather than noise from small variations.

In [846]:
df['CompensationRank'], bins = pd.qcut(df['ConvertedCompYearly'], 5, labels=[1, 2, 3, 4, 5], retbins=True)
print(bins) 

[ 47000.  98000. 125000. 150000. 187000. 550000.]


We will visualize the compensation distribution using color coding to represent the different ranks.

In [847]:
# Create a histogram with Plotly
fig = px.histogram(
    df,
    x='ConvertedCompYearly',
    color= 'CompensationRank',
    title='Compensation Distribution',
    labels={'ConvertedCompYearly': 'Total Compensation'},
    nbins=100,  
    width=1200
)

# Update the layout for better appearance
fig.update_layout(
    xaxis_title='Compensation',
    yaxis_title='Frequency',
    bargap=0.1,  # Gap between bars
    showlegend=False
)


fig.show()

In [848]:
# Check for any values that might have been incorrectly assigned
# incorrect_assignments = df[(df['ConvertedCompYearly'] > bins[4]) & (df['CompensationRank'] != 5)]
# len(incorrect_assignments)

Next, we will one-hot encode the dataset, as most of the columns are categorical in nature. To achieve this, I will reuse a function specifically designed for processing categorical columns.

In [849]:
def process_columns_with_dummies(df, dummy_na=False):
    """
    This function processes the specified columns in the given DataFrame by splitting the values,
    creating dummy variables for each unique value in each column, and then combining these dummy variables back into the original DataFrame.

    Parameters:
    df (pd.DataFrame): The original DataFrame containing the specified columns.
    columns (list of str): A list of column names to be processed.

    Returns:
    pd.DataFrame: The modified DataFrame with dummy variables for each unique value in the specified columns.
    """
    
    columns = df.select_dtypes(include=['object'])
    
    
    for column in columns:
        
        # Remove square brackets and split the specified column into a list. 
        temp_df = df[column].str.replace('\\[|\\]', '', regex=True).str.split(';')
        
        # Explode to multiple rows
        temp_df = temp_df.explode().reset_index()
        
        # Create dummy variables for the specified column
        dummies_df = pd.get_dummies(temp_df[column], dummy_na=dummy_na, drop_first=True, prefix=column, prefix_sep='_')

        # Combine dummy variables with the original dataframe
        combined_df = temp_df[['index']].join(dummies_df).groupby('index').sum()

        # Join the dummy variables back to the original dataframe
        df = df.drop(column, axis=1).join(combined_df)
    
    return df

orig_df = df.copy()
df = process_columns_with_dummies(df)
df.shape

(3774, 959)

In [850]:
# Checking for missing values
column_info(df).sort_values(by='Missing Values',ascending=False)

,Column,Dtype,Missing Values,missing_values_pct,Cardinality
3,WorkExp,float64,1059,0.280604,50
1,YearsCodePro,float64,72,0.019078,49
0,YearsCode,float64,11,0.002915,50
5,CompensationRank,category,0,0.000000,5
6,Age_25-34 years old,int64,0,0.000000,2
...,...,...,...,...,...
954,Industry_Wholesale,int64,0,0.000000,2
955,SurveyLength_Too long,int64,0,0.000000,2
956,SurveyLength_Too short,int64,0,0.000000,2
957,SurveyEase_Easy,int64,0,0.000000,2


The WorkExp column has 28% missing values. To address this, I will use the dropna method to drop these rows. I expect a sufficient number of samples will remain for analysis.

In [851]:
df = df.dropna()
df.shape


(2662, 959)

I believe the data preparation is complete for now, and we can proceed to extract initial insights using a correlation matrix. Let's identify which features are highly correlated with higher compensation ranks.

In [852]:
# Calculate the correlation matrix
correlation_matrix = df.drop(['ConvertedCompYearly', 'CompTotal'], axis=1).corr()
# Extract correlations with the target variable 'ConvertedCompYearly'
df_correlations = correlation_matrix['CompensationRank'].sort_values(ascending=False)
# Convert df_correlations from a Series to a DataFrame
df_correlations = df_correlations.to_frame().reset_index()
# Rename the columns to be more descriptive
df_correlations.columns = ['Feature', 'Correlation']
# Add color column
df_correlations['Color'] = ['Positive' if val > 0 else 'Negative' for val in df_correlations['Correlation']]
# Drop na values
df_correlations = df_correlations.dropna()
# Drop target feature
df_correlations = df_correlations.drop(index=0)



In [853]:
# Display top 10
df_correlations.head(10)

,Feature,Correlation,Color
1,ProfessionalTech_Observability tools,0.344806,Positive
2,ProfessionalTech_Automated testing,0.320127,Positive
3,YearsCodePro,0.317351,Positive
4,YearsCode,0.286997,Positive
5,ProfessionalTech_Continuous integration (CI) a...,0.280751,Positive
6,OpSysProfessional use_MacOS,0.278161,Positive
7,WorkExp,0.273360,Positive
8,ProfessionalTech_Microservices,0.249963,Positive
9,OfficeStackSyncHaveWorkedWith_Slack,0.240541,Positive
10,ToolsTechHaveWorkedWith_Terraform,0.224021,Positive


In [854]:
# Displat bottom 10
df_correlations.tail(10)


,Feature,Correlation,Color
944,LanguageHaveWorkedWith_PHP,-0.146187,Negative
945,LanguageHaveWorkedWith_C#,-0.147355,Negative
946,RemoteWork_In-person,-0.151856,Negative
947,WebframeHaveWorkedWith_jQuery,-0.158238,Negative
948,OpSysPersonal use_Windows,-0.164303,Negative
949,NEWCollabToolsHaveWorkedWith_Visual Studio,-0.165832,Negative
950,DatabaseHaveWorkedWith_Microsoft SQL Server,-0.193907,Negative
951,OfficeStackSyncHaveWorkedWith_Microsoft Teams,-0.197093,Negative
952,ProfessionalTech_None of these,-0.221370,Negative
953,OpSysProfessional use_Windows,-0.264054,Negative


I will create a simple function to render a bar chart. I prefer using Plotly over more traditional choices like Matplotlib and Seaborn.

In [855]:
def plot_bar_chart(df, title, x, y, order):

    # Creating the bar chart with Plotly
    fig = px.bar(
        df,
        x=x,
        y=y,
        orientation='h',
        title=title,
        width=1200,
        height=500,
        color_discrete_map={'Positive': 'green', 'Negative': 'red'},
        color = 'Color'
        
    )

    # Updating layout for better appearance
    fig.update_layout(
        yaxis={'categoryorder':f'total {order}'},
        title_x=0.5,
        margin=dict(l=50, r=50, t=50, b=40, pad=20),
        
    )
    
    fig.update_xaxes(
        title_standoff = 50)

    fig.update_yaxes(
        title_standoff = 50)
    
    fig.update_traces(width=0.3)

    fig.show()

In [856]:
plot_bar_chart(df_correlations.head(10), 'Positive Correlation', 'Correlation', 'Feature', 'ascending')


In [857]:
plot_bar_chart(df_correlations.tail(10), 'Negative Correlation', 'Correlation', 'Feature', 'descending')

That's very interesting. Now, let's see what insights we can gain from linear regression. I will use Lasso regression because it usually what works best me and often gives me better R² scores and mean squared error (MSE) compared to standard linear regression.

In [858]:
# Drop columns that are not needed for prediction
X = df.drop(['ConvertedCompYearly', 'CompTotal', 'CompensationRank'], axis=1)

# Select the target variable and reshape for scaling
y = df['CompensationRank'].values.reshape(-1, 1)  # Reshape for scaling

# Initialize the scaler for features and target
scaler_X = StandardScaler()  # Scaler for feature variables
scaler_y = StandardScaler()  # Scaler for the target variable

# Fit and transform the features
X_scaled = scaler_X.fit_transform(X)  # Scale the feature variables

# Fit and transform the target
y_scaled = scaler_y.fit_transform(y)  # Scale the target variable

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_scaled, test_size=0.2, random_state=42
)  # Split data into training and testing sets



We will evaluate the model using Mean Squared Error (MSE) and R-squared metrics to assess its performance. This analysis helps us understand the significant features contributing to compensation variations.

In [859]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error


# Lasso Regression
lasso = Lasso(alpha=0.02)
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)

# Calculate MSE
mse_lasso = mean_squared_error(y_test, y_pred_lasso)

# Calculate R-squared
r2_lasso = r2_score(y_test, y_pred_lasso)

print("Lasso Regression MSE:", mse_lasso)
print("Lasso Regression R-squared:", r2_lasso)


Lasso Regression MSE: 0.5177987812270305
Lasso Regression R-squared: 0.45798439723586903


An MSE of 0.5 and an R² score of 0.45 are modest results, but they offer a solid starting point. By examining the model coefficients, we can gain valuable insights into the importance of specific features.

We are now ready to present the results:

In [860]:
# Extracting feature importances from Lasso regression
lasso_importance = lasso.coef_

# Creating a DataFrame with feature names and their corresponding importances
df_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': lasso_importance
})

df_importances['AbsImportance'] = df_importances['Importance'].abs()
df_importances['Color'] = ['Positive' if val > 0 else 'Negative' for val in df_importances['Importance']]


df_importances = df_importances.sort_values(by='Importance', ascending=False)

plot_bar_chart(df_importances.head(10),'Positive Impact','Importance','Feature','ascending')




In [861]:
plot_bar_chart(df_importances.tail(10), 'Negative Impact','Importance','Feature','descending')

In [862]:
# Extract the top 10 features from each dataframe
top10_df1 = df_importances.head(10)['Feature'].tolist()
top10_df2 = df_correlations.head(10)['Feature'].tolist()

# Find common features
common_features = set(top10_df1).intersection(set(top10_df2))
common_features

{'OfficeStackSyncHaveWorkedWith_Slack',
 'OpSysProfessional use_MacOS',
 'ProfessionalTech_Automated testing',
 'ProfessionalTech_Observability tools',
 'ToolsTechHaveWorkedWith_Terraform',
 'YearsCodePro'}

In [863]:
# Extract the bot 10 features from each dataframe
bot10_df1 = df_importances.tail(10)['Feature'].tolist()
bot10_df2 = df_correlations.tail(10)['Feature'].tolist()

# Find common features
common_features = set(bot10_df1).intersection(set(bot10_df2))
common_features

{'LanguageHaveWorkedWith_PHP',
 'OfficeStackSyncHaveWorkedWith_Microsoft Teams',
 'OpSysProfessional use_Windows'}

An interesting insight from both the correlation matrix and the coefficient importance list is that using Windows for professional purposes is moderately correlated with lower compensation.

Let's examine the average compensation of engineers who use Windows for work and compare it to those who use macOS.

In [864]:
pro_condition = orig_df['OpSysProfessional use'].isin(['MacOS', 'Windows'])
pro_opsys_df = orig_df[pro_condition].groupby('OpSysProfessional use').agg(
    Count=('OpSysProfessional use', 'size'),
    MeanComp=('ConvertedCompYearly', 'mean')

).reset_index().rename(columns={'OpSysProfessional use': 'OS'}).sort_values(by='MeanComp', ascending=True)

pro_opsys_df['Percentage Change'] = pro_opsys_df['MeanComp'].pct_change() * 100
pro_opsys_df['MeanComp'] = pro_opsys_df['MeanComp'].apply(lambda x: f'${x:,.1f}')


pro_opsys_df.head(10)


,OS,Count,MeanComp,Percentage Change
1,Windows,809,"$125,429.3",NaN
0,MacOS,947,"$168,240.2",34.131513


<div style="border: 2px solid #000; padding: 15px; border-radius: 8px; background-color: #000; color: #fff; text-align: center;">

### Compensation Difference

<span style="font-size: 48px; color: #32CD32;">+34%</span>

**macOS users** earn more than **Windows users** for professional use.

</div>


### 4. My next question is - What about personal use?

We will calculate the average compensation for Windows users and compare it to macOS users, this time focusing on personal use.

In [865]:
per_condition = orig_df['OpSysPersonal use'].isin(['MacOS', 'Windows'])
per_opsys_df = orig_df[per_condition].groupby('OpSysPersonal use').agg(
    Count=('OpSysPersonal use', 'size'),
    MeanComp=('ConvertedCompYearly', 'mean')
).reset_index().rename(columns={'OpSysPersonal use': 'OS'}).sort_values(by='MeanComp', ascending=True)

per_opsys_df['Percentage Change'] = per_opsys_df['MeanComp'].pct_change() * 100
per_opsys_df['MeanComp'] = per_opsys_df['MeanComp'].apply(lambda x: f'${x:,.1f}')

per_opsys_df.head(10)

,OS,Count,MeanComp,Percentage Change
1,Windows,821,"$134,331.5",NaN
0,MacOS,671,"$166,982.9",24.306627


<div style="border: 2px solid #000; padding: 15px; border-radius: 8px; background-color: #000; color: #fff; text-align: center;">

### Compensation Difference

<span style="font-size: 48px; color: #32CD32;">+24%</span>

**macOS users** earn more than **Windows users** for personal use.

</div>

<div style="max-width: 1000px;">

### Conclusion

In our examination of the 2023 Stack Overflow Developer Survey, we delved into key aspects of developer compensation for full-time employees in the US. Here are the notable findings:

1. **Developer Type Compensation**: As expected, senior executives command the highest median salaries. Interestingly, developer experience engineers hold the second spot in compensation rankings, surpassing other specialized roles. Contrary to expectations, full-stack developers find themselves lower on the compensation scale, with a notable $70,000 gap compared to developer experience engineers.

2. **Factors Influencing Full-Stack Developer Compensation**: Our analysis identified key variables that affect earnings among full-stack developers. Notably, the operating system used professionally plays a significant role. Developers using macOS typically reported higher salaries than their counterparts using Windows.

3. **Impact of Operating System on Compensation**: Across both professional and personal use, macOS users consistently reported higher compensation than those using Windows. This trend highlights the potential financial benefits of choosing specific operating systems within the tech industry.

</div>
